In [1]:
import json
import os
import shutil
from tqdm import tqdm  # tqdm 추가

# 경로 설정
source_image_dir = '../../dataset/train'  # 원래 이미지들이 저장된 디렉토리
train_image_dir = '../../dataset/yolo_train80/images'
val_image_dir = '../../dataset/yolo_val20/images'
train_label_dir = '../../dataset/yolo_train80/labels'
val_label_dir = '../../dataset/yolo_val20/labels'

# 디렉토리 생성 (존재하지 않으면 생성)
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

# 이미지 크기 설정 (데이터셋에 맞게 설정)
image_width = 1024  # 실제 이미지 가로 크기
image_height = 1024  # 실제 이미지 세로 크기

# YOLO 형식으로 변환 및 이미지 복사
def coco_to_yolo(json_path, image_dir, label_dir):
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # 이미지 ID와 해당 이미지 파일명을 매핑
    image_id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}
    image_id_to_size = {img['id']: (img['width'], img['height']) for img in coco_data['images']}

    # 어노테이션 데이터를 처리
    for ann in tqdm(coco_data['annotations'], desc=f"Processing {json_path}"):
        image_id = ann['image_id']
        category_id = ann['category_id']  # class_id
        bbox = ann['bbox']  # COCO 형식의 bbox (xmin, ymin, width, height)

        # COCO 형식의 bbox를 YOLO 형식으로 변환
        xmin, ymin, width, height = bbox
        img_width, img_height = image_id_to_size[image_id]
        center_x = (xmin + width / 2) / img_width
        center_y = (ymin + height / 2) / img_height
        norm_width = width / img_width
        norm_height = height / img_height

        # YOLO 라벨 파일 경로 생성 및 디렉토리 확인
        label_file_path = os.path.join(label_dir, f'{image_id_to_filename[image_id].split(".")[0].split("/")[1]}.txt')
        os.makedirs(os.path.dirname(label_file_path), exist_ok=True)  # 디렉토리 생성

        # # YOLO 라벨 파일 작성
        with open(label_file_path, 'a') as yolo_file:  # 같은 이미지에 여러 개체가 있을 수 있으므로 append 모드
            yolo_file.write(f'{category_id} {center_x} {center_y} {norm_width} {norm_height}\n')

        
        # # 이미지 파일 복사
        source_image_path = os.path.join(source_image_dir, image_id_to_filename[image_id].split("/")[1])
        target_image_path = os.path.join(image_dir, image_id_to_filename[image_id].split("/")[1])
        if not os.path.exists(target_image_path):  # 이미 복사된 이미지라면 다시 복사하지 않음
            shutil.copy(source_image_path, target_image_path)


# train.json에서 라벨 파일 생성 및 이미지 복사
coco_to_yolo("../../dataset/train_80.json", train_image_dir, train_label_dir)

# val.json에서 라벨 파일 생성 및 이미지 복사
coco_to_yolo("../../dataset/val_20.json", val_image_dir, val_label_dir)

print("이미지 복사 및 YOLO 라벨 파일 생성이 완료되었습니다.")

Processing ../../dataset/val_20.json: 100%|██████████| 14270/14270 [00:01<00:00, 11110.82it/s]

이미지 복사 및 YOLO 라벨 파일 생성이 완료되었습니다.
